In [6]:
import numpy as np
import pandas as pd

class matrix:
    def __init__(self, filename=None):
        self.array_2d = None
        if filename:
            self.load_from_csv(filename)

    def load_from_csv(self, filename):
        data = pd.read_csv(filename)
        self.array_2d = data.to_numpy()

    def standardise(self):
        for j in range(self.array_2d.shape[1]):
            col = self.array_2d[:, j]
            mean = np.mean(col)
            max_val = np.max(col)
            min_val = np.min(col)
            self.array_2d[:, j] = (col - mean) / (max_val - min_val)

    def get_distance(self, other_matrix, row_i):
        distances = np.linalg.norm(self.array_2d[row_i] - other_matrix.array_2d, axis=1)
        return matrix.from_array(distances.reshape(-1, 1))

    def get_weighted_distance(self, other_matrix, weights, row_i):
        diff = self.array_2d[row_i] - other_matrix.array_2d
        weighted_diff = np.sqrt(np.sum(weights.array_2d * (diff ** 2), axis=1))
        return matrix.from_array(weighted_diff.reshape(-1, 1))

    def get_count_frequency(self):
        if self.array_2d.shape[1] != 1:
            return 0
        unique, counts = np.unique(self.array_2d, return_counts=True)
        return dict(zip(unique, counts))

    @staticmethod
    def from_array(array):
        mat = matrix()
        mat.array_2d = array
        return mat
    
def get_initial_weights(m):
    weights = np.random.rand(1, m)
    weights /= weights.sum()
    return matrix.from_array(weights)

def get_centroids(data_matrix, S_matrix, K):
    centroids = np.zeros((K, data_matrix.array_2d.shape[1]))
    for k in range(K):
        cluster_points = data_matrix.array_2d[S_matrix.array_2d[:, 0] == k + 1]
        if len(cluster_points) > 0:
            centroids[k] = np.mean(cluster_points, axis=0)
    return matrix.from_array(centroids)

def get_separation_within(data_matrix, centroids, S_matrix, K):
    separation_within = np.zeros((1, data_matrix.array_2d.shape[1]))
    for k in range(K):
        for i in range(data_matrix.array_2d.shape[0]):
            if S_matrix.array_2d[i, 0] == k + 1:
                diff = data_matrix.array_2d[i] - centroids.array_2d[k]
                separation_within += np.square(diff)
    return matrix.from_array(separation_within)

def get_separation_between(data_matrix, centroids, S_matrix, K):
    separation_between = np.zeros((1, data_matrix.array_2d.shape[1]))
    for k in range(K):
        cluster_size = np.sum(S_matrix.array_2d[:, 0] == k + 1)
        if cluster_size > 0:
            mean_diff = centroids.array_2d[k] - np.mean(data_matrix.array_2d, axis=0)
            separation_between += cluster_size * np.square(mean_diff)
    return matrix.from_array(separation_between)

def get_groups(data_matrix, K):
    n = data_matrix.array_2d.shape[0]
    m = data_matrix.array_2d.shape[1]
    
    centroids = data_matrix.array_2d[np.random.choice(n, K, replace=False)]
    S = np.zeros((n, 1))
    weights = get_initial_weights(m)
    
    for _ in range(10):
        for i in range(n):
            distances = np.linalg.norm(data_matrix.array_2d[i] - centroids, axis=1)
            S[i, 0] = np.argmin(distances) + 1

        new_centroids = get_centroids(data_matrix, matrix.from_array(S), K).array_2d
        if np.all(centroids == new_centroids):
            break
        centroids = new_centroids

    return matrix.from_array(S)

def get_new_weights(data_matrix, centroids, weights, S_matrix, K):
    a = get_separation_within(data_matrix, centroids, S_matrix, K)
    b = get_separation_between(data_matrix, centroids, S_matrix, K)
    new_weights = (weights.array_2d + b.array_2d / a.array_2d) / (1 + np.sum(b.array_2d / a.array_2d))
    return matrix.from_array(new_weights)

def run_test():
    m = matrix('Data (2).csv')
    for k in range(2, 11):
        S = get_groups(m, k)
        print(str(k) + '=' + str(S.get_count_frequency()))


if __name__ == "__main__":
    run_test()

2={1.0: 54, 2.0: 123}
3={1.0: 62, 2.0: 69, 3.0: 46}
4={1.0: 29, 2.0: 60, 3.0: 20, 4.0: 68}
5={1.0: 39, 2.0: 20, 3.0: 46, 4.0: 29, 5.0: 43}
6={1.0: 40, 2.0: 20, 3.0: 31, 4.0: 19, 5.0: 41, 6.0: 26}
7={1.0: 28, 2.0: 27, 3.0: 22, 4.0: 24, 5.0: 20, 6.0: 19, 7.0: 37}
8={1.0: 32, 2.0: 23, 3.0: 40, 4.0: 17, 5.0: 6, 6.0: 23, 7.0: 19, 8.0: 17}
9={1.0: 23, 2.0: 27, 3.0: 14, 4.0: 12, 5.0: 21, 6.0: 17, 7.0: 22, 8.0: 6, 9.0: 35}
10={1.0: 10, 2.0: 4, 3.0: 19, 4.0: 30, 5.0: 16, 6.0: 14, 7.0: 13, 8.0: 25, 9.0: 26, 10.0: 20}
